# Ensemble

In [ ]:
import pandas as pd
from tqdm import tqdm

### Ensemble Setting

In [ ]:
single_model_indexs=[[7,8,9,23,3,0,5,10],[7,13,9,17,15,19,22,14],[7,8,9,4,3,10,11],[7,8,9,4,3,10,11,12,13,14,5,0,1],[7,8,9,4,3,10,11,12,13,14,5,15,16,17,19,6,20,21,18,22],[7,24,16,24,13,11,20,10,9,8],[7,8,9,4,3,10,11,12,13,14,5,0],[7,8,9,4,3,2,22,6],[9,25,7,26,24,23],[7,12,11,14,22,17],[],[],[],[],[],[2]]
ensemble_indexs=[[],[],[],[],[],[],[],[],[],[2],[1,2,9],[5,9],[1,9,2],[0,11,10,6,3,12,4,7],[0,10,3,12,4],[14,12]]
ensemble_methods=[['Mean'],['Mean'],['Mean'],['Mean'],['Mean'],['Mean'],['Mean'],['Mean'],['Mean'],['Mean'],['Mean'],['Mean'],['Median'],['Median'],['Median,INT'],['Median']]

###  Ensemble with given setting

In [ ]:
sample_submission = pd.read_csv('./dataset/sample_submission.csv')
for i in tqdm(range(len(single_model_indexs))):
    sample_submission['target']=0

    # Mean 앙상블
    if ensemble_methods[i]=='Mean':
        for s_ind in single_model_indexs[i]:
            sample_submission['target'] += pd.read_csv(f'./model_prediction_{s_ind}.csv')['target']
        for e_ind in ensemble_indexs[i]:
            sample_submission['target'] += pd.read_csv(f'./ensemble_{e_ind}.csv')['target']
        sample_submission['target'] /= len(single_model_indexs[i])+len(ensemble_indexs[i])

    # Median 앙상블
    else:
        cnt=0
        for s_ind in single_model_indexs[i]:
            sample_submission[f'sample{cnt}'] = pd.read_csv(f'./model_prediction_{s_ind}.csv')['target']
            cnt+=1
        for e_ind in ensemble_indexs[i]:
            sample_submission[f'sample{cnt}'] = pd.read_csv(f'./ensemble_{e_ind}.csv')['target']
            cnt+=1
        sample_submission['target'] = sample_submission.filter(like='sample').apply(lambda x: x.median(), axis=1)
        
        sample_submission = sample_submission.drop([f'sample{j}' for j in range(len(single_model_indexs[i])+len(ensemble_indexs[i]))],axis=1)
        # 앙상블 결과 int로 변환하기
        if ensemble_methods[i]=='Median,INT':
            sample_submission['target'] = sample_submission.apply(lambda x: x['target']//1,axis=1)

    # 앙상블 결과 저장
    if i==len(single_model_indexs)-1:
        sample_submission.to_csv('./ensemble_submission.csv',index=False)
    else:
        sample_submission.to_csv(f'./ensemble_{i}.csv',index=False)
